# 输出提示词重写

之前，内置的提示词是：

```markdown

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
        "disease": list  // 疾病名称实体
        "symptom": list  // 疾病症状实体
        "drug": list  // 药物名称实体
}

```

\```



这会导致总体提示词中英文混杂，并且比较笨的模型可能会把注释一起输出了，这里做出改进。

新提示词：

```markdown
请从以下文本中，抽取出实体信息，并按json格式返回，json包含首尾的 "```json" 和 "```":
以下是字段含义和类型，要求保留所有字段：
disease 字段，表示：疾病名称实体，类型为：list
symptom 字段，表示：疾病症状实体，类型为：list
drug 字段，表示：药物名称实体，类型为：list
```

In [1]:
import shutup
shutup.please()

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

load_dotenv()
llm = ChatOpenAI(model_name='qwen2:7b', temperature=0)

需要重写解析器：

In [3]:
def structured_output_parser(response_schemas):
    text = '''
    请从以下文本中，抽取出实体信息，并按json格式返回，json包含首尾的 "```json" 和 "```":
    以下是字段含义和类型，要求保留所有字段：
    '''
    for schema in response_schemas:
        text += schema.name + ' 字段，表示：' + schema.description + '，类型为：' + schema.type + '\n'
    return text

response_schemas = [
    ResponseSchema(type='list', name='disease', description='疾病名称实体'),
    ResponseSchema(type='list', name='symptom', description='疾病症状实体'),
    ResponseSchema(type='list', name='drug', description='药物名称实体'),
]

format_instructions = structured_output_parser(response_schemas)
format_instructions

'\n    请从以下文本中，抽取出实体信息，并按json格式返回，json包含首尾的 "```json" 和 "```":\n    以下是字段含义和类型，要求保留所有字段：\n    disease 字段，表示：疾病名称实体，类型为：list\nsymptom 字段，表示：疾病症状实体，类型为：list\ndrug 字段，表示：药物名称实体，类型为：list\n'

In [4]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

template = '''
{format_instructions}
------------
用户输入：{input}
------------
输出：
'''

prompt = PromptTemplate(
    template=template,
    partial_variables={'format_instructions': format_instructions},
    input_variables=['input']
)

from langchain.chains import LLMChain

chain = LLMChain(
    llm = llm,
    prompt = prompt
)

llm_output = chain.run(input='感冒是一种什么病？会导致咳嗽吗？')

output = output_parser.parse(llm_output)
print(output, type(output))

{'disease': ['感冒'], 'symptom': ['咳嗽'], 'drug': []} <class 'dict'>
